In [2]:
#tensorflow versiyonunu kontrol ediyor
import tensorflow as tf
print(tf.__version__)

2.10.0


In [3]:
#kütüphanelerin yüklenmesi
import matplotlib.pyplot as plt
import pandas as pd
import os
import librosa
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 



In [4]:
#Librosa kütüphanesini incelemeye başlayalım. 
#Burada Librosa kütüphanesi ile karmaşık bir ses sinyali dosyasını yani stereo dosyasını mono sinyale dönüştürür. 
#Tek boyutlu olur ve dijital hale getir.

In [5]:
#ses sinyali okuyalım
audio_file_path='audio_file_path.vav' #veriyi eklediğimiz kısım

librosa_audio_data, librosa_sample_rate = librosa.load(audio_file_path)

C:\Users\Monster\AppData\Local\Temp\ipykernel_7256\306815564.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  librosa_audio_data, librosa_sample_rate = librosa.load(audio_file_path)
C:\Users\Monster\anaconda3\envs\tensorflow\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: 'audio_file_path.vav'

In [ ]:
print(librosa_audio_data)

In [ ]:
# Librosa ses verilerini çizelim
# 1 kanallı orijinal ses
plt.figure(figsize=(12, 4))
plt.plot(librosa_audio_data)
plt.show()

In [ ]:
#Aslında örneğimiz stereo, scipy kullanarak görebilirsiniz
from scipy.io import wavfile as wav
wave_sample_rate, wave_audio = wav.read(audio_file_path)

In [ ]:
wave_audio

In [ ]:
# 2 kanallı orijinal ses ve görselleştirme işlemi
plt.figure(figsize=(12, 4))
plt.plot(wave_audio)
plt.show()

In [ ]:
# çıktılarda gözüken gibi mavi kısımlar içerisindeki farklı ses blokları temsil eder.

ÖZELLİK ÇIKARIMI

Burada ses örneklerinden Mel-Frekans Cepstral Katsayılarını (MFCC) kullanacağız. MFCC, pencere boyutu boyunca frekans dağılımını özetler, böylece sesin hem frekansını hem de zaman özelliklerini analiz etmek mümkündür. Bu sesli temsiller, sınıflandırma için özellikleri tanımlamamıza izin verecektir.

In [ ]:
#n_mfcc: number of MFCCs to return 
mfccs = librosa.feature.mfcc(y=librosa_audio_data, sr=librosa_sample_rate, n_mfcc=40)   
print(mfccs.shape)

In [ ]:
mfccs

In [ ]:
#Veri kümesindeki her ses dosyası için MFCC'leri çıkaracağız.

audio_dataset_path='audio_dataset_path' #audio ses dosyalarının olduğu yol
metadata=pd.read_csv('cvs_path') #csv yolu
metadata.head()

In [ ]:
def features_extractor(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [ ]:
# Şimdi her bir ses dosyasını tekrarlıyoruz ve özellikleri ayıklıyoruz 
# Mel-Frekans Cepstral Katsayılarını kullanarak
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [21]:
# Öznitelik çıkarım'ı Pandas veri çerçevesine dönüştüreceğiz
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.520657, 52.00812, ...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036358, ...",children_playing
4,"[-446.60352, 113.68541, -52.402206, 60.302044,...",children_playing


In [22]:
# Daha sonra veri setini bağımsız ve bağımlı veri setine ayırdık
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [23]:
X.shape

(8732, 40)

In [24]:
y.shape

array([[-2.1735526e+02,  7.0223381e+01, -1.3038527e+02, ...,
        -1.6930529e+00, -6.1698377e-01,  3.8600534e-01],
       [-4.2409818e+02,  1.0934077e+02, -5.2919525e+01, ...,
         5.3489321e-01, -5.4468721e-01,  4.4632095e-01],
       [-4.5879114e+02,  1.2138419e+02, -4.6520657e+01, ...,
         2.0768483e+00,  1.6962965e+00, -9.6140957e-01],
       ...,
       [-3.0388824e+02,  1.1135945e+02, -4.5941563e+01, ...,
        -3.0292380e+00,  2.7170298e+00,  7.6197419e+00],
       [-3.4411008e+02,  1.2545021e+02, -5.4903442e+01, ...,
        -7.9082437e+00, -1.6414586e+00,  5.6668444e+00],
       [-3.1560281e+02,  9.4854805e+01, -3.7222340e+01, ...,
         6.1386454e-01, -1.1449189e+01, -6.0105853e+00]], dtype=float32)

In [25]:
# Modelimizde (1'ler ve 0'lar) çıktı sınıfları için bir sıcak kodlanmış değere ihtiyacımız olduğundan, Etiket Kodlaması yapmalıyız.

labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [26]:
y[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [27]:
# Veri setini Train and Test olarak ayırdık

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [28]:
X_train

array([[-1.31104706e+02,  1.12505905e+02, -2.25746956e+01, ...,
         3.24665213e+00, -1.36902380e+00,  2.75575471e+00],
       [-1.36703424e+01,  9.10850830e+01, -7.79273319e+00, ...,
        -3.25305080e+00, -5.27745247e+00, -1.55697179e+00],
       [-4.98715439e+01,  2.65352994e-01, -2.05009365e+01, ...,
         2.85459447e+00, -1.60920465e+00,  3.52480578e+00],
       ...,
       [-4.27012360e+02,  9.26230469e+01,  3.12939739e+00, ...,
         7.42641389e-01,  7.33490944e-01,  7.11009085e-01],
       [-1.45754608e+02,  1.36265778e+02, -3.35155182e+01, ...,
         1.46811950e+00, -2.00917006e+00, -8.82181883e-01],
       [-4.21031342e+02,  2.10654541e+02,  3.49066067e+00, ...,
        -5.38886738e+00, -3.37136054e+00, -1.56651151e+00]], dtype=float32)

In [29]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
X_train.shape

(6985, 40)

In [31]:
X_test.shape

(1747, 40)

In [32]:
y_train.shape

(6985, 10)

In [33]:
y_test.shape

(1747, 10)

Evrişimli Sinir Ağları (CNN) Modeli Oluşturma ve Modelimizi Veri Kümesi ile Eğitme.

In [34]:
# Kaç sınıfımız var? Onu kendi modelimizde kullanmalıyız.
num_labels = 10

In [35]:
# CNN modelimiz

model=Sequential()
# 1. hidden layer
model.add(Dense(125,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 2. hidden layer
model.add(Dense(250))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# 3. hidden layer
model.add(Dense(125))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 125)               5125      
                                                                 
 activation (Activation)     (None, 125)               0         
                                                                 
 dropout (Dropout)           (None, 125)               0         
                                                                 
 dense_1 (Dense)             (None, 250)               31500     
                                                                 
 activation_1 (Activation)   (None, 250)               0         
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                                 
 dense_2 (Dense)             (None, 125)               3

In [37]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [38]:
# Modeli eğitmek

epochscount = 300
num_batch_size = 32

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=epochscount, validation_data=(X_test, y_test), verbose=1)


Epoch 1/300
219/219 [==============================] - 1s 4ms/step - loss: 10.2766 - accuracy: 0.1223 - val_loss: 2.2934 - val_accuracy: 0.1185
Epoch 2/300
219/219 [==============================] - 1s 3ms/step - loss: 2.4814 - accuracy: 0.1165 - val_loss: 2.2828 - val_accuracy: 0.1162
Epoch 3/300
219/219 [==============================] - 1s 3ms/step - loss: 2.3235 - accuracy: 0.1162 - val_loss: 2.2577 - val_accuracy: 0.1339
Epoch 4/300
219/219 [==============================] - 1s 3ms/step - loss: 2.2593 - accuracy: 0.1373 - val_loss: 2.2001 - val_accuracy: 0.1740
Epoch 5/300
219/219 [==============================] - 1s 2ms/step - loss: 2.2340 - accuracy: 0.1589 - val_loss: 2.1640 - val_accuracy: 0.1797
Epoch 6/300
219/219 [==============================] - 1s 2ms/step - loss: 2.1914 - accuracy: 0.1702 - val_loss: 2.0908 - val_accuracy: 0.2473
Epoch 7/300
219/219 [==============================] - 1s 2ms/step - loss: 2.1437 - accuracy: 0.1951 - val_loss: 1.9672 - val_accuracy: 0.275

Epoch 58/300
219/219 [==============================] - 1s 3ms/step - loss: 1.0110 - accuracy: 0.6639 - val_loss: 0.8261 - val_accuracy: 0.7539
Epoch 59/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9909 - accuracy: 0.6659 - val_loss: 0.8310 - val_accuracy: 0.7510
Epoch 60/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9879 - accuracy: 0.6737 - val_loss: 0.8021 - val_accuracy: 0.7401
Epoch 61/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9971 - accuracy: 0.6743 - val_loss: 0.8218 - val_accuracy: 0.7441
Epoch 62/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9992 - accuracy: 0.6729 - val_loss: 0.8189 - val_accuracy: 0.7436
Epoch 63/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9943 - accuracy: 0.6684 - val_loss: 0.8265 - val_accuracy: 0.7390
Epoch 64/300
219/219 [==============================] - 1s 3ms/step - loss: 0.9824 - accuracy: 0.6690 - val_loss: 0.8087 - val_accuracy:

Epoch 115/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8634 - accuracy: 0.7108 - val_loss: 0.6972 - val_accuracy: 0.7888
Epoch 116/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8692 - accuracy: 0.7183 - val_loss: 0.6801 - val_accuracy: 0.7859
Epoch 117/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8560 - accuracy: 0.7127 - val_loss: 0.7173 - val_accuracy: 0.7728
Epoch 118/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8439 - accuracy: 0.7233 - val_loss: 0.6826 - val_accuracy: 0.7848
Epoch 119/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8470 - accuracy: 0.7193 - val_loss: 0.7012 - val_accuracy: 0.7813
Epoch 120/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8418 - accuracy: 0.7187 - val_loss: 0.6826 - val_accuracy: 0.7922
Epoch 121/300
219/219 [==============================] - 1s 3ms/step - loss: 0.8549 - accuracy: 0.7190 - val_loss: 0.7142 - val_ac

219/219 [==============================] - 1s 3ms/step - loss: 0.7969 - accuracy: 0.7367 - val_loss: 0.5989 - val_accuracy: 0.8231
Epoch 228/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7640 - accuracy: 0.7500 - val_loss: 0.5964 - val_accuracy: 0.8157
Epoch 229/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7812 - accuracy: 0.7426 - val_loss: 0.6139 - val_accuracy: 0.8111
Epoch 230/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7699 - accuracy: 0.7457 - val_loss: 0.6105 - val_accuracy: 0.8088
Epoch 231/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7734 - accuracy: 0.7459 - val_loss: 0.5882 - val_accuracy: 0.8197
Epoch 232/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7856 - accuracy: 0.7402 - val_loss: 0.5982 - val_accuracy: 0.8111
Epoch 233/300
219/219 [==============================] - 1s 3ms/step - loss: 0.7744 - accuracy: 0.7466 - val_loss: 0.5959 - val_accuracy: 0.8191

In [39]:
validation_test_set_accuracy = model.evaluate(X_test,y_test,verbose=0)
print(validation_test_set_accuracy[1])

0.8202633261680603


In [40]:
X_test[1]

array([-466.17957  ,    1.0950246,  -34.01389  ,   35.33935  ,
        -14.88148  ,  -19.12843  ,   -0.581684 ,  -16.130579 ,
        -21.339075 ,    7.673634 ,  -29.16449  ,  -18.950253 ,
         -2.9579992,   -8.162329 ,  -15.153101 ,   -6.604805 ,
         -7.5685983,    9.340646 ,   14.4331   ,   21.934181 ,
         20.861397 ,    1.3340123,  -19.228804 ,   -4.630231 ,
         -1.0564743,    3.215267 ,   -6.984281 ,  -16.414577 ,
        -10.0286455,   13.009956 ,    0.5334608,  -23.843391 ,
        -15.267321 ,    9.245734 ,   10.367627 ,   -0.5832011,
         -1.2624055,   17.700016 ,   13.847463 ,   -5.1862826],
      dtype=float32)

In [41]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

55/55 [==============================] - 0s 1ms/step


CNN Modelimizi Kullanarak Bir Ses Dosyasının Sınıfını Tahmin Ediyoruz.

Önce yeni ses ortamı olarak işliyoruz ve sonraki sınıfı tahmin ediyoruz.

In [42]:
filename="wav_path"
sound_signal, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=sound_signal, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

In [43]:
print(mfccs_scaled_features)

[-2.6935144e+02  1.3180406e+02  3.0552094e+00 -1.5418642e+01
 -4.5053989e+01  8.1400518e+00 -2.3114010e+01 -2.0470648e+01
 -2.4931124e+01 -8.9026871e+00 -1.0712789e+00  1.3080616e+00
 -3.2105761e+00 -3.1079106e+00 -1.6007302e+00  6.1797256e+00
 -6.4077549e-02 -6.0688586e+00 -5.3680668e+00 -1.9135537e+00
 -1.0260819e+01 -6.0929441e+00 -7.8891764e+00 -4.2310629e+00
 -2.6515894e+00  8.2413852e-01  9.9788368e-01 -1.2166636e+00
 -9.5165503e-01  3.0775521e+00 -3.9112416e-01 -3.5094411e+00
 -1.4894108e+00  1.0855072e+00 -1.9442009e-02 -2.2774961e+00
 -2.3359878e+00 -2.4188461e+00 -2.3379452e+00 -8.8834047e-01]


In [44]:
mfccs_scaled_features = mfccs_scaled_features.reshape(1,-1)

In [45]:
mfccs_scaled_features.shape

(1, 40)

In [46]:
print(mfccs_scaled_features)

[[-2.6935144e+02  1.3180406e+02  3.0552094e+00 -1.5418642e+01
  -4.5053989e+01  8.1400518e+00 -2.3114010e+01 -2.0470648e+01
  -2.4931124e+01 -8.9026871e+00 -1.0712789e+00  1.3080616e+00
  -3.2105761e+00 -3.1079106e+00 -1.6007302e+00  6.1797256e+00
  -6.4077549e-02 -6.0688586e+00 -5.3680668e+00 -1.9135537e+00
  -1.0260819e+01 -6.0929441e+00 -7.8891764e+00 -4.2310629e+00
  -2.6515894e+00  8.2413852e-01  9.9788368e-01 -1.2166636e+00
  -9.5165503e-01  3.0775521e+00 -3.9112416e-01 -3.5094411e+00
  -1.4894108e+00  1.0855072e+00 -1.9442009e-02 -2.2774961e+00
  -2.3359878e+00 -2.4188461e+00 -2.3379452e+00 -8.8834047e-01]]


In [47]:
print(mfccs_scaled_features.shape)

(1, 40)


In [48]:
result_array = model.predict(mfccs_scaled_features)

1/1 [==============================] - 0s 28ms/step


In [49]:
result_array

array([[1.41048930e-38, 1.11740809e-22, 9.58587682e-25, 1.00000000e+00,
        1.39567028e-21, 1.10433581e-28, 4.41576636e-12, 0.00000000e+00,
        1.04882581e-17, 1.06594054e-16]], dtype=float32)

In [50]:
result_classes = ["air_conditioner","car_horn","children_playing","dog_bark","drilling", "engine_idling", "gun_shot", "jackhammer", "siren", "street_music"]

result = np.argmax(result_array[0])
print(result_classes[result]) 

dog_bark
